# ML.Net - BinaryClassifierFieldAwareFactorizationMachine

In [1]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML"  
    
//Install XPlot package
#r "nuget:XPlot.Plotly"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.0

Installed package XPlot.Plotly version 3.0.1

In [2]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML.Transforms;
using static Microsoft.ML.DataOperationsCatalog;
using XPlot.Plotly;

In [3]:
//Step 1: Declare Path to Training Data Set
static readonly string _trainingDataPath = Path.Combine(Environment.CurrentDirectory, "./Datasets/yelp_labelled/yelp_labelled.csv");

//Declare Path to Saving the Trained MLModel
static readonly string _modelPath = Path.Combine(Environment.CurrentDirectory, "./Datasets/yelp_labelled/BinaryClassifierFieldAwareFactorizationMachine/Model.zip");

In [4]:
class SentimentPredictionOutput : SentimentInputData
{
    [ColumnName("PredictedLabel")]
    public bool Prediction { get; set; }
    
    public float Probability { get; set; }
    
    public float Score { get; set; }
}

class SentimentInputData
{
    [LoadColumn(0)]
    public string SentimentText;

    [LoadColumn(1), ColumnName("Label")]
    public bool Sentiment;
}

In [5]:
// Declare classifySentiment Method
public static void classifySentiment()
{
    //Step 2: Instantiate MLContext() object
    MLContext mLContext = new MLContext();

    //Load Training Data and Split the Training Data into 80% for Training and 20% for Evaluation
    //Step 3: Create IDataView() object using mlContext.Data to load the Training Data Set
    IDataView trainingDataView = mLContext.Data.LoadFromTextFile<SentimentInputData>(_trainingDataPath, hasHeader: false, separatorChar: ',');

    //Step 4: Split the Training Data Automatically into 80% for Training and 20% for Evaluation
    TrainTestData trainTestData = mLContext.Data.TrainTestSplit(trainingDataView, testFraction: 0.2);

    //Create pipeline() object and Training MLModel
    //Step 5: Create pipeline() object
    var dataPreparationPipeline = mLContext.Transforms.Text.FeaturizeText(outputColumnName: "Features", inputColumnName: nameof(SentimentInputData.SentimentText));

    //Step 6: Create fieldAwareFactorizationMachineTrainer() using dataPreparationPipeline.Append() 
    var fieldAwareFactorizationMachineTrainer = dataPreparationPipeline.Append(mLContext.BinaryClassification.Trainers.FieldAwareFactorizationMachine(labelColumnName: "Label", featureColumnName: "Features"));

    //Step 6: Training MLModel using trainTestData.TrainSet
    var model = fieldAwareFactorizationMachineTrainer.Fit(trainTestData.TrainSet);

    // Evaluation and Assessment of the Trained MLModel
    //Step 7: Evaluation using trainTestData.TestSet
    IDataView predictionsForEvaluationDataView = model.Transform(trainTestData.TestSet);

    //Step 8: Evaluation Metrics for MLModel Assessment
    CalibratedBinaryClassificationMetrics metrics = mLContext.BinaryClassification.Evaluate(predictionsForEvaluationDataView, labelColumnName: "Label");

    // Step 9: Display the Performance Metrics of the MLModel
    Console.WriteLine();
    Console.WriteLine("----------------Binary Classification Trainer Algorithm: FieldAwareFactorizationMachine-------");
    Console.WriteLine();
    Console.WriteLine("  Assessing MLModel Quality using Evaluation Metrics");
    Console.WriteLine();
    Console.WriteLine($"  Accuracy: {metrics.Accuracy:P2}");
    Console.WriteLine($"  F1 Score: {metrics.F1Score:P2}");
    Console.WriteLine();
    Console.WriteLine($"  Area Under ROC Curve: {metrics.AreaUnderRocCurve:P2}");
    Console.WriteLine($"  Area Under Precision-Recall Curve: {metrics.AreaUnderPrecisionRecallCurve:P2}");
    Console.WriteLine();
    Console.WriteLine($"  Log Loss: {metrics.LogLoss.ToString()}");
    Console.WriteLine($"  Log Loss Reduction: {metrics.LogLossReduction.ToString()}");
    Console.WriteLine();
    Console.WriteLine($"  Confusion Matrix: {metrics.ConfusionMatrix.GetFormattedConfusionTable().ToString()}");
    Console.WriteLine();
    Console.WriteLine($"  Positive Precision: {metrics.PositivePrecision.ToString()}");
    Console.WriteLine($"  Positive Recall: {metrics.PositiveRecall.ToString() }");
    Console.WriteLine($"  Negative Precision: {metrics.NegativePrecision.ToString()}");
    Console.WriteLine($"  Negative Recall: {metrics.NegativeRecall.ToString()}");
    Console.WriteLine($"  Positive Precision: {metrics.PositivePrecision.ToString()}");
    Console.WriteLine();
    Console.WriteLine("---------------End of Model Quality Metrics------------");


    //Step 10: Create new Sample Text for the  MLModel To Predict the Sentiment of New Text Data
    SentimentInputData sampleSentimentText = new SentimentInputData
    {
        SentimentText = "I love this restaurant"
    };

    // Step 11: Create  predictionFunction()
    var predictionFunction = mLContext.Model.CreatePredictionEngine<SentimentInputData, SentimentPredictionOutput>(model);

    // Step 12: Use predictionFunction to predict sentiment
    var predictionResult = predictionFunction.Predict(sampleSentimentText);

    // Step 13: Write the sentimentResult  to the Output Display
    Console.WriteLine();
    Console.WriteLine("-------Trained Model Prediction using New Sentiment Text Data-------");
    Console.WriteLine($"Prdiction Sentence: {predictionResult.SentimentText}|" +
        $"Predicted Sentiment: {(Convert.ToBoolean(predictionResult.Prediction) ? "Positive" : "Negative")}|" +
        $"Probability: {predictionResult.Probability}");

    //Step 14: Save Trained MLModel
    using (var fileStream = new FileStream(_modelPath, FileMode.Create, FileAccess.Write, FileShare.Write))
    {
        mLContext.Model.Save(model, trainingDataView.Schema, fileStream);
        Console.WriteLine();
        Console.WriteLine("Saved Trained MLModel using the specified Path");
        Console.WriteLine();
        Console.WriteLine("-----------------End of Output----------------------------------");
    }
}

In [6]:
classifySentiment()


----------------Binary Classification Trainer Algorithm: FieldAwareFactorizationMachine-------

  Assessing MLModel Quality using Evaluation Metrics

  Accuracy: 65,95%
  F1 Score: 59,87%

  Area Under ROC Curve: 72,37%
  Area Under Precision-Recall Curve: 63,50%

  Log Loss: 0,9301955461796418
  Log Loss Reduction: 0,05734894169736966

  Confusion Matrix: TEST POSITIVE RATIO:	0,4324 (80,0/(80,0+105,0))
Confusion table
          ||======================
PREDICTED || positive | negative | Recall
TRUTH     ||======================
 positive ||       47 |       33 | 0,5875
 negative ||       30 |       75 | 0,7143
          ||======================
Precision ||   0,6104 |   0,6944 |


  Positive Precision: 0,6103896103896104
  Positive Recall: 0,5875
  Negative Precision: 0,6944444444444444
  Negative Recall: 0,7142857142857143
  Positive Precision: 0,6103896103896104

---------------End of Model Quality Metrics------------

-------Trained Model Prediction using New Sentiment Text Data--